<h1>Grabbing all Cryptos</h1>

In [1]:
#Main Dependencies
import pandas as pd
import requests
from tkinter import *
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np

In [2]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\citiz\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [ ]:
#Establish URL
url = 'https://coinmarketcap.com/'

In [ ]:
tokens = []

for i in range(0, 43, 1):
    loop_url = f'https://coinmarketcap.com/?page={i}'
    browser.visit(loop_url)
    
    for j in range(10):
        browser.execute_script("window.scrollBy(0,1000);")
        time.sleep(.5)
        
    html = browser.html
    
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find('table', class_='cmc-table')
    trs = table.find_all('tr')

    for tr in trs:

        token = {}

        tds = tr.find_all('td')

        try:
            if tds[2].p is not None:
                coin_name = tds[2].p.text
                coin_code = tds[2].div.div.div.p.text
            else:
                coin_name = tds[2].find_all('span')[1].text
                coin_code = tds[2].find_all('span')[2].text

            coin_price = float(tds[3].text.replace(',','').strip('$'))
            coin_24h = float(tds[4].text.strip('%'))
            coin_7d = float(tds[5].text.strip('%'))
            
            try:
                coin_market_cap = float(tds[6].text.replace(',','').strip('$'))
            except Exception as e:
                coin_market_cap = None
                print(f'Warning: row [{i}] is missing market cap.')                

            # from Kick: use this as an example to isolate volume, which is sometimes blank
            try:
                coin_volume_usd = float(tds[7].find_all('p')[0].text.replace(',','').strip('$'))
                coin_volume_crypto_list = tds[7].find_all('p')[1].text.replace(',','').strip('$').split()
                coin_volume = float(coin_volume_crypto_list[0])
                coin_volume_token = coin_volume_crypto_list[1]
            except Exception as e:
                i = trs.index(tr)
                print(f'Warning: row [{i}] is missing volume.')
                coin_volume_usd = None
                coin_volume_crypto_list = None
                coin_volume = None
                coin_volume_token = None

            try:
                coin_circulating_supply_list = tds[8].text.replace(',','').strip('$').split()
                coin_circulating_supply = float(coin_circulating_supply_list[0])
                coin_circulating_supply_token = coin_circulating_supply_list[1]
            except Exception as e:
                print(f'Warning: row [{i}] is missing circulating supply.')                
                coin_circulating_supply_list = None
                coin_circulating_supply = None
                coin_circulating_supply_token = None
                

            # set up dictionary record
            token['token'] = coin_code
            token['name'] = coin_name
            token['price'] = coin_price
            token['24h'] = coin_24h
            token['7d'] = coin_7d
            token['market_cap'] = coin_market_cap
            token['volume_usd'] = coin_volume_usd
            token['volume'] = coin_volume
            token['circulating_supply'] = coin_circulating_supply

            # add to the list of dictionaries
            tokens.append(token)

        except Exception as e:
            i = trs.index(tr)
            #print(f'Error with index [{i}] of the tr.')
            continue
            
browser.quit()

In [ ]:
#Create DataFrame
token = pd.DataFrame(tokens)
#token

In [ ]:
#Extract Master array of NAMES ONLY for all Cryptos
coins = token['name'].values
#coins

In [ ]:
#Extract Master array of NAMES ONLY for all Cryptos
codes = token['token'].values
codes

<h1>Exchange Data - Database<h1>

In [ ]:
#Grab All URLS
links = []
for c in range(0,len(coins),1):
    lowercase = codes[c].lower()
    for r in ((" ", "-"), (".", "-")):
        lowercase = lowercase.replace(*r)
        exchange = f'https://coinmarketcap.com/currencies/{lowercase}/'
        
    #Appending
    links.append(exchange)

#Print Final Dictionary
#links

In [ ]:
#LOGO PHOTO
logos = []
for c in range(0,len(coins),1):
    lowercase = coins[c].lower()
    lc_code = codes[c].lower()
    for r in ((" ", "-"), (".", "-")):
        lowercase = lowercase.replace(*r)
        snap = f'https://cryptologos.cc/logos/{lowercase}-{lc_code}-logo.png?v=010'
        
    #Appending
    logos.append(snap)

#Print Final Dictionary
#logos

In [ ]:
dollars = []

for l in range(0,len(links),1):
    
    dollar = {}
        
    stats = links[l]
    analysis = requests.get(stats)
    broth = BeautifulSoup(analysis.text, 'html.parser')
    tables = broth.findAll('table')
    
    try:
        first_table_scrape = tables[0]
        first_table_tds = first_table_scrape.find_all('td')
    except:
        continue
    try:
        second_table_scrape = tables[1]
        second_table_tds = second_table_scrape.find_all('td')
    except:
        continue
    try:
        third_table_scrape = tables[2]
        third_table_tds = third_table_scrape.find_all('td')
    except:
        continue
    try:
        fourth_table_scrape = tables[3]
        fourth_table_tds = fourth_table_scrape.find_all('td')
    except:
        continue
    try:
        fifth_table_scrape = tables[4]
        fifth_table_tds = fifth_table_scrape.find_all('td')
    except:
        continue

    texts = broth.select('div.about___1OuKY')
    code = broth.findAll('div', class_='sc-16r8icm-0 fCTazK nameHeader___27HU_')[0].find_all('small')[0].getText()
        
    try:
        price = first_table_tds[0].get_text()
    except: 
        price = None
    try:
        pricechange24hr = first_table_tds[1].find_all('span')[0].get_text()
        percentchange24hr = first_table_tds[1].find_all('span')[1].get_text()
    except:
        pricechange24hr = None
        percentchange24hr = None
    try:
        low24hr = first_table_tds[2].find_all('div')[0].get_text().split(' /')[0]
        high24hr = first_table_tds[2].find_all('div')[1].get_text().split(' /')[0]
    except:
        low24hr = None
        high24hr = None
    try:
        tradingvolume = first_table_tds[3].find_all('span')[0].get_text()
        tradingvolumepercdelta = first_table_tds[3].find_all('span')[1].get_text()
    except:
        tradingvolume = None
        tradingvolumepercdelta = None
    try:
        marketdominance = first_table_tds[4].find_all('span')[0].get_text()
        marketrank = first_table_tds[5].get_text().split('#')[1]
        marketcap = second_table_tds[0].find_all('span')[0].get_text()
        fullydiluted = second_table_tds[1].find_all('span')[0].get_text()
    except:
        marketdominance = None
        marketrank = None
        marketcap = None
        fullydiluted = None
    
    try:
        yestlow = third_table_tds[0].find_all('div')[0].get_text().split(' /')[0]
        yesthigh = third_table_tds[0].find_all('div')[1].get_text().split(' /')[0]
        yestopen = third_table_tds[1].find_all('div')[0].get_text().split(' /')[0]
        yestclose = third_table_tds[1].find_all('div')[1].get_text().split(' /')[0]
        yestvol = third_table_tds[3].get_text()
    except:
        yestlow = None
        yesthigh = None
        yestopen = None
        yestclose = None
        yestvol = None

    try:
        d7low = fourth_table_tds[0].find_all('div')[0].get_text().split(' /')[0]
        d7high = fourth_table_tds[0].find_all('div')[1].get_text().split(' /')[0]
    except:
        d7low = None
        d7high = None
    try:
        d30low = fourth_table_tds[1].find_all('div')[0].get_text().split(' /')[0]
        d30high = fourth_table_tds[1].find_all('div')[1].get_text().split(' /')[0]  
    except:
        d30low = None
        d30high = None
    try:
        d90low = fourth_table_tds[2].find_all('div')[0].get_text().split(' /')[0]
        d90high = fourth_table_tds[2].find_all('div')[1].get_text().split(' /')[0] 
    except:
        d90low = None
        d90high = None
    try:
        wk52low = fourth_table_tds[3].find_all('div')[0].get_text().split(' /')[0]
        wk52high = fourth_table_tds[3].find_all('div')[1].get_text().split(' /')[0]
    except:
        wk52low = None
        wk52high = None
    try:
        alltimehigh = fourth_table_tds[4].find_all('span')[0].get_text()
        alltimehighperc = fourth_table_tds[4].find_all('span')[1].get_text()
        alltimelow = fourth_table_tds[5].find_all('span')[0].get_text()
        alltimelowperc = fourth_table_tds[5].find_all('span')[1].get_text()
    except:
        alltimehigh = None
        alltimehighperc = None
        alltimelow = None
        alltimelowperc = None
    try:
        roi = fourth_table_tds[6].findAll('p')[0].getText()
    except:
        roi = None
    try:
        circsupp = fifth_table_tds[0].get_text()
        totsupp = fifth_table_tds[1].get_text()
        mxsupp = fifth_table_tds[2].get_text()
    except:
        circsupp = None
        totsupp = None
        mxsupp = None
    intro1 = texts[1].find_all('p')[0].get_text()
    intro2 = texts[1].find_all('p')[1].get_text()
    fullintro = f'{intro1}{intro2}'

    dollar['Code'] = code
    dollar['Price'] = price
    dollar['24hr Price Change'] = pricechange24hr
    dollar['24hr % Change'] = percentchange24hr
    dollar['24hr Low'] = low24hr
    dollar['24hr High'] = high24hr
    dollar['24hr Trading Volume'] = tradingvolume
    dollar['24hr Trading Volume % Change'] = tradingvolumepercdelta
    dollar['Market Dominance'] = marketdominance
    dollar['Market Rank'] = marketrank
    dollar['Market Cap'] = marketcap
    dollar['Fully Diluted Market Cap'] = fullydiluted
    dollar['Yesterday Low'] = yestlow
    dollar['Yesterday High'] = yesthigh
    dollar['Yesterday Open'] = yestopen
    dollar['Yesterday Close'] = yestclose
    dollar['Yesterday Volume'] = yestvol
    dollar['7 Day Low'] = d7low
    dollar['7 Day High'] = d7high
    dollar['30 Day Low'] = d30low
    dollar['30 Day High'] = d30high
    dollar['90 Day Low'] = d90low
    dollar['90 Day High'] = d90high
    dollar['52 Week Low'] = wk52low
    dollar['52 Week High'] = wk52high
    dollar['All Time High'] = alltimehigh
    dollar['All Time High %'] = alltimehighperc
    dollar['All Time Low'] = alltimelow
    dollar['All Time Low %'] = alltimelowperc
    dollar['Return on Investment'] = roi
    dollar['Cirulating Supply'] = circsupp
    dollar['Total Supply'] = totsupp
    dollar['Max Supply'] = mxsupp
    dollar["About"] = fullintro
    dollar['Link'] = stats

    #Appending
    dollars.append(dollar)

#Print Final Dictionary
#dollars
all_exchange = pd.DataFrame(dollars)
all_exchange

In [ ]:
#Export to CSV
all_exchange.to_csv('ExchangeInfo.csv', index=False, header=True)

In [ ]:
#Export to JSON
import csv
import json

def make_json(csvFilePath, jsonFilePath):

    data = {}
     
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        for rows in csvReader:

            key = rows['Code']
            data[key] = rows

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         
csvFilePath = r'ExchangeInfo.csv'
jsonFilePath = r'ExchangeInfo.json'
 
make_json(csvFilePath, jsonFilePath)

<h1>Historical Data - DataBase</h1>

In [3]:
#NEW URL - Creating Date List
dates = 'https://coinmarketcap.com/historical/'

In [4]:
#Cooking in the Kitchen
select = requests.get(dates)
pho = BeautifulSoup(select.text, 'html.parser')
#pho

In [5]:
#Pull history of all dates
links = pho.find_all('div',class_='cmc-bottom-margin-2x')[0].find_all('a',class_='historical-link cmc-link')
#links

In [ ]:
#Create DataFrame
dates = []

for l in range(36,len(links),1):
    
    date = {}
    
    nums=links[l].get('href').split("/")[2]
    
    dates.append(nums)
    
#dates

In [ ]:
#Grab All URLS
links = []
for d in range(0,len(dates),1):
    history = f'https://coinmarketcap.com/historical/{dates[d]}/'
        
    #Appending
    links.append(history)

#Print Final Dictionary
#links

In [ ]:
logs = []

for l in range(0,len(links),1):
    story = requests.get(links[l])
    chowder = BeautifulSoup(story.text, 'html.parser')
    exhibits = chowder.findAll('tbody')
    try:
        itinerary = exhibits[0]
    except:
        print('Scraping Error. Data Not Available')
    itinerary_trs = itinerary.find_all('tr')
    for w in range(0,len(itinerary),1):

        tds = itinerary_trs[w].find_all('td')

        log = {}

        rank = tds[0].find_all('div')[0].get_text()
        name = tds[1].find_all('a')[0].getText()
        symbol = tds[2].find_all('div')[0].get_text()
        marketcap = tds[3].find_all('p')[0].getText()
        price = tds[4].find_all('a')[0].getText()
        circsupp = tds[5].find_all('div')[0].get_text()
        vol24h = tds[6].find_all('a')[0].getText()
        p1h = tds[7].find_all('div')[0].get_text()
        p24h = tds[8].find_all('div')[0].get_text()
        p7d = tds[9].find_all('div')[0].get_text()

        log['Date'] = links[l].split('/')[4]
        log['Rank'] = rank
        log['Name'] = name
        log['Symbol'] = symbol
        log['Market Cap'] = marketcap
        log['Price'] = price
        log['Circulating Supply'] = circsupp
        log['1 Hour Percent Change'] = p1h
        log['24 Hour Percent Change'] = p24h
        log['7 Day Percent Change'] = p7d


        logs.append(log)

#logs
textbook = pd.DataFrame(logs)
textbook['Date'] = pd.to_datetime(textbook['Date'], format='%Y%m%d',errors='coerce')
#textbook

In [ ]:
#Export to CSV
textbook.to_csv('History.csv', index=False, header=True)

In [ ]:
#Export to JSON
import csv
import json

def make_json(csvFilePath, jsonFilePath):

    data = {}
     
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        for rows in csvReader:

            key = rows['Date']
            data[key] = rows

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         
csvFilePath = r'History.csv'
jsonFilePath = r'History.json'
 
make_json(csvFilePath, jsonFilePath)